# AoA方式(二次元)

In [926]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import default_rng
from lmfit.models import SineModel
from scipy.optimize import curve_fit
from typing import Optional
import sympy as sp
from sympy.plotting import plot3d
from components import point2d as p2
from components import point3d as p3
from components import antennaPair as antp
from components import antenna as ant
import itertools
%matplotlib inline

In [927]:
c = 2.998e8 #光の速度
f = 920.4e6 #周波数
WAVE_LENGTH = c / f * 100 #理論波長
THETA_M = 34 * np.pi / 180
MAX_L = WAVE_LENGTH / (2 * np.sin(THETA_M))
L = 25
R = 45
DATA_LENGTH = 50
print(f'波長λ:{WAVE_LENGTH}, Lの最大値:{MAX_L}')

波長λ:32.57279443720122, Lの最大値:29.124828154140907


In [929]:
DATA_FILE_PATH_LIST=[
    '..//data//triangulation_GTag_0.csv',
    '..//data//triangulation_GTag_p20.csv',
    '..//data//triangulation_GTag_m20.csv',
    '..//data//triangulation_GTag_p30.csv',
    '..//data//triangulation_GTag_m30.csv',
    '..//data//aoa_localizaton_m25_m25.csv'
]

In [930]:
TARGET_TAG = 'E200 470F C1A0 6023 A8E6 010F'

In [931]:
ANTENNA_PAIR_LIST = {
    'Pair1' : [ant.Antenna(point=p2.Point2d(-12.5, -200), frequency=920.4, number=4, theta_m=34),ant.Antenna(point=p2.Point2d(12.5, -200), frequency=920.4, number=6, theta_m=34)],
    'Pair2' : [ant.Antenna(point=p2.Point2d(-12.5, 200), frequency=920.4, number=4, theta_m=34),ant.Antenna(point=p2.Point2d(12.5, 200), frequency=920.4, number=6, theta_m=34)]
}

In [ ]:
ANGLE_RANGE = range(int(-(len(DATA_FILE_PATH_LIST)-1)/2*5), int((len(DATA_FILE_PATH_LIST)-1)/2*5+5), 5)
R1_LIST = [np.sqrt(pow(R, 2) + pow((L/2), 2) + R * L * np.sin((i) * np.pi / 180)) for i in ANGLE_RANGE]
R2_LIST = [np.sqrt(pow(R, 2) + pow((L/2), 2) - R * L * np.sin((i) * np.pi / 180)) for i in ANGLE_RANGE]
ACTUAL_PHASE1 = [(-4 * np.pi / WAVE_LENGTH * R1_LIST[i]) % (2 * np.pi) for i in range(0,len(DATA_FILE_PATH_LIST))]
ACTUAL_PHASE2 = [(-4 * np.pi / WAVE_LENGTH * R2_LIST[i]) % (2 * np.pi) for i in range(0,len(DATA_FILE_PATH_LIST))]

In [932]:
class RfidDf:
    def __init__(self, csv_file:str, phi0_file:str, antenna_pair_list:dict, target_tag:str):
        self.df=pd.read_csv(csv_file)
        phi0_df = pd.read_csv(phi0_file)
        self.data_size = len(self.df)
        self.antenna_pair_list = antenna_pair_list
        self.target_tag = target_tag
        self.df = self.df[self.df['Tag ID'] == self.target_tag]
        self.df['Antenna Pair No.'] = self.df['Antenna No.'].apply(self.__get_antenna_pair_number)
        self.df['Phase'] = self.df['Phase'].astype(float)
        self.df['RSSI'] = self.df['RSSI'].astype(float)
        self.df = self.df.drop(['Tag ID'], axis=1).set_index(['Antenna Pair No.', 'Antenna No.', 'Data Number'])
        phi0_df = phi0_df[phi0_df['Tag ID'] == self.target_tag]
        phi0_df['Antenna Pair No.'] = phi0_df['Antenna No.'].apply(self.__get_antenna_pair_number)
        phi0_df['Phase'] = phi0_df['Phase'].astype(float)
        phi0_df['RSSI'] = phi0_df['RSSI'].astype(float)
        phi0_df = phi0_df.drop(['Tag ID'], axis=1).set_index(['Antenna Pair No.', 'Antenna No.', 'Data Number'])
        self.phi0 = self.get_phi0(phi0_df=phi0_df)
    
    def __get_antenna_pair_number(self, antenna_number:int) -> str:
        for k in self.antenna_pair_list:
            for i in self.antenna_pair_list[k]:
                if i.number == antenna_number:
                    return k

    def __get_phase_diff(self, phase1:float, phase2:float, rssi1:float, rssi2:float) -> float:
        delta_phi = (phase1-phase2)-(self.phi0[0]-self.phi0[1])
        if (delta_phi < 0) & (rssi1 > rssi2):
            delta_phi = delta_phi + 2 * np.pi
        if (delta_phi >= 0) & (rssi1 <= rssi2):
            delta_phi = delta_phi - 2 * np.pi
        return delta_phi

    def __cul_angle_from_phase(self, phase1:float, phase2:float, rssi1:float, rssi2:float) -> float:
        return np.arcsin(-WAVE_LENGTH / (4 * np.pi * L) * self.__get_phase_diff(phase1, phase2, rssi1, rssi2)) * 180 / np.pi

    def get_phi0(self, phi0_df:pd.DataFrame) -> list:
        ant1_phi0=0
        ant2_phi0 = 0
        for pair in self.antenna_pair_list:
            ant1 = self.antenna_pair_list[pair][0]
            ant2 = self.antenna_pair_list[pair][1]
            ant1_phi0 = phi0_df.sort_index().loc[(pair, ant1.number)]['Phase'].mean() + ACTUAL_PHASE1[8]
            ant2_phi0 = phi0_df.sort_index().loc[(pair, ant2.number)]['Phase'].mean() + ACTUAL_PHASE1[8]
        return [ant1_phi0, ant2_phi0]

    def __get_angles_from_phase(self) -> dict:
        result = {}
        for pair in self.antenna_pair_list:
            ant1 = self.antenna_pair_list[pair][0]
            ant2 = self.antenna_pair_list[pair][1]
            phase_diff_mean = 0
            angle_mean = 0
            for i in range(0, DATA_LENGTH):
                ant1_phase = (self.df.xs((pair, ant1.number, i))['Phase']) % (2 * np.pi)
                ant2_phase = (self.df.xs((pair, ant2.number, i))['Phase']) % (2 * np.pi)
                ant1_rssi = self.df.xs((pair, ant1.number, i))['RSSI']
                ant2_rssi = self.df.xs((pair, ant2.number, i))['RSSI']
                phase_diff_mean = (phase_diff_mean + self.__get_phase_diff(ant1_phase, ant2_phase, ant1_rssi, ant2_rssi)) / 2
                angle_mean = (angle_mean + self.__cul_angle_from_phase(ant1_phase, ant2_phase, ant1_rssi, ant2_rssi)) / 2
            result[pair]["phase_diff"] = phase_diff_mean
            result[pair]["angle_mean"] = angle_mean
        return result

    def get_coordinate(self) -> dict:
        angles = self.__get_angles_from_phase()
        pair1 = self.antenna_pair_list["Pair1"]
        pair2 = self.antenna_pair_list["Pair2"]
        pair1_point = p2.Point2d(x=(pair1[0].point.x + pair1[1].point.x) / 2, y=(pair1[0].point.y + pair1[1].point.y) / 2)
        pair2_point = p2.Point2d(x=(pair2[0].point.x + pair2[1].point.x) / 2, y=(pair2[0].point.y + pair2[1].point.y) / 2)
        pair1_angle = angles["Pair1"]["angle_mean"]
        pair2_angle = angles['Pair2']["angle_mean"]
        result = {
            "x" : np.tan(pair1_angle) * np.tan(pair2_angle) / (np.tan(pair1_angle) - np.tan(pair2_angle)) * (pair1_point - pair2_point),
            "y" : (pair1_point * np.tan(pair1_angle) - pair2_point * np.tan(pair2_angle)) / (np.tan(pair1_angle) - np.tan(pair2_angle))
        }

In [933]:
triangulation = RfidDf(
    csv_file=DATA_FILE_PATH_LIST[5],
    phi0_file=DATA_FILE_PATH_LIST[8],
    antenna_pair_list=ANTENNA_PAIR_LIST,
    target_tag=TARGET_TAG,
)

In [934]:
result = triangulation.get_coordinate()
print(result)